In this live coding exercise, we will implement Newton's method for unconstrained optimization.\
We will then use the implemented method to try and find the minimum of Rosenbrock function expressed as:
$$
f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (x_i - 1)^2\right]
$$\
Rosenbrock function is a standard test function used for testing gradient-descent based algorithms.\
It can be defined in $n$-dimensions arbitrarily. It is unimodal and the global minimum of the function is 0 at $x_1, \ldots, x_n = 1$.\

In 2-D Rosenbrock function is expressed as:
$$
f(x, y) = (1 - x)^2 + 100(y - x^2)^2
$$

In [1]:
# Import relevant libraries
import numpy as np

In [2]:
# A Python function implementing Newton's method
def newtons_method(grad_f, hessian, x0, epsilon=1e-6, k_max=100):
    """
    Implements Newton's method for optimization.
    
    Parameters:
    grad_f (function): A function that computes the gradient of f at a point x.
    hessian (function): A function that computes the Hessian of f at a point x.
    x0 (numpy.ndarray): Initial guess for the solution.
    epsilon (float): Tolerance for stopping criteria (default: 1e-6).
    k_max (int): Maximum number of iterations (default: 100).
    
    Returns:
    numpy.ndarray: The final value of x that minimizes the function.
    """
    # Initialize variables
    x = x0
    k = 1
    delta = np.full_like(x0, np.inf)  # Initialize delta as large values

    # Main optimization loop
    while np.linalg.norm(delta) > epsilon and k <= k_max:
        # Compute the Newton step
        grad = grad_f(x)  # Gradient at current x
        hess = hessian(x)  # Hessian at current x
        
        # Solve for the Newton step Δ: H(x) * Δ = grad_f(x)
        delta = np.linalg.solve(hess, grad)
        
        # Update x
        x = x - delta
        
        # Increment iteration counter
        k += 1

    return x

#### Gradient of $f(x, y)$
The gradient $\nabla f(x, y)$ is the vector of partial derivatives of $f(x, y)$ with respect to $x$ and $y$:
$$
\nabla f(x, y) = \begin{bmatrix} 
\frac{\partial f}{\partial x} \\
\frac{\partial f}{\partial y}
\end{bmatrix}
$$

##### Partial Derivative with Respect to $x$:
$$
\frac{\partial f}{\partial x} = \frac{\partial}{\partial x} \left[ (1 - x)^2 + 100(y - x^2)^2 \right]
$$

- First term: $(1 - x)^2$:
$$
\frac{\partial}{\partial x}(1 - x)^2 = -2(1 - x)
$$

- Second term: $100(y - x^2)^2$:
$$
\frac{\partial}{\partial x} 100(y - x^2)^2 = 100 \cdot 2(y - x^2) \cdot (-2x) = -400x(y - x^2)
$$

Combining both terms:
$$
\frac{\partial f}{\partial x} = -2(1 - x) - 400x(y - x^2)
$$

##### Partial Derivative with Respect to $y$:
$$
\frac{\partial f}{\partial y} = \frac{\partial}{\partial y} \left[ (1 - x)^2 + 100(y - x^2)^2 \right]
$$

- First term: $(1 - x)^2$:
$$
\frac{\partial}{\partial y}(1 - x)^2 = 0 \quad (\text{no dependence on } y)
$$

- Second term: $100(y - x^2)^2$:
$$
\frac{\partial}{\partial y} 100(y - x^2)^2 = 100 \cdot 2(y - x^2) = 200(y - x^2)
$$

Thus:
$$
\frac{\partial f}{\partial y} = 200(y - x^2)
$$

---

#### Final Gradient Expression:
$$
\nabla f(x, y) = \begin{bmatrix}
-2(1 - x) - 400x(y - x^2) \\
200(y - x^2)
\end{bmatrix}
$$

This gradient is used in the Newton's method implementation to compute the Newton step $\Delta$.

In [3]:
# Define the Rosenbrock function's gradient
def grad_rosenbrock(x):
    """
    Gradient of the 2D Rosenbrock function.
    """
    x1, x2 = x
    grad_x1 = -2 * (1 - x1) - 400 * x1 * (x2 - x1**2)
    grad_x2 = 200 * (x2 - x1**2)
    return np.array([grad_x1, grad_x2])

##### Second Partial Derivative with Respect to $x$:
The second derivative $\frac{\partial^2 f}{\partial x^2}$ is:
$$
\frac{\partial^2 f}{\partial x^2} = \frac{\partial}{\partial x} \left[ -2(1 - x) - 400x(y - x^2) \right]
$$

- First term: $-2(1 - x)$:
$$
\frac{\partial}{\partial x}[-2(1 - x)] = 2
$$

- Second term: $-400x(y - x^2)$:
  - First, expand $\frac{\partial}{\partial x}[-400x(y - x^2)]$:
  $$ 
  \frac{\partial}{\partial x}[-400x(y - x^2)] = \frac{\partial}{\partial x}[-400xy + 400x^3] 
  $$
  - For $-400xy$: 
  $$
  \frac{\partial}{\partial x}[-400xy] = -400y
  $$
  - For $400x^3$: 
  $$
  \frac{\partial}{\partial x}[400x^3] = 1200x^2
  $$

Combining the terms:
$$
\frac{\partial^2 f}{\partial x^2} = 2 - 400y + 1200x^2
$$

---

##### Mixed Partial Derivative $\frac{\partial^2 f}{\partial x \partial y}$:
The mixed partial derivative $\frac{\partial^2 f}{\partial x \partial y}$ is:
$$
\frac{\partial^2 f}{\partial x \partial y} = \frac{\partial}{\partial y} \left[ -2(1 - x) - 400x(y - x^2) \right]
$$

- First term: $-2(1 - x)$:
$$
\frac{\partial}{\partial y}[-2(1 - x)] = 0 \quad (\text{no dependence on } y)
$$

- Second term: $-400x(y - x^2)$:
$$
\frac{\partial}{\partial y}[-400x(y - x^2)] = -400x
$$

Thus:
$$
\frac{\partial^2 f}{\partial x \partial y} = -400x
$$

---

##### Second Partial Derivative with Respect to $y$:
The second derivative $\frac{\partial^2 f}{\partial y^2}$ is:
$$
\frac{\partial^2 f}{\partial y^2} = \frac{\partial}{\partial y} \left[ 200(y - x^2) \right]
$$

- First term: $200(y - x^2)$:
$$
\frac{\partial}{\partial y}[200(y - x^2)] = 200
$$

Thus:
$$
\frac{\partial^2 f}{\partial y^2} = 200
$$

---

#### Final Hessian Matrix:
Combining all terms, the Hessian matrix $\mathbf{H}$ is:
$$
\mathbf{H} = 
\begin{bmatrix}
2 - 400y + 1200x^2 & -400x \\
-400x & 200
\end{bmatrix}
$$

This matrix is used in Newton's method to compute the Newton step $\Delta$.

In [4]:
# Define the Rosenbrock function's Hessian
def hessian_rosenbrock(x):
    """
    Hessian of the 2D Rosenbrock function.
    """
    x1, x2 = x
    h11 = 2 - 400 * (x2 - x1**2) + 800 * x1**2
    h12 = -400 * x1
    h21 = -400 * x1
    h22 = 200
    return np.array([[h11, h12], [h21, h22]])

Let us provide the guess as $x=0$ and $y=0$.

In [5]:
x0 = np.array([0, 0])

Let us run the Newton's method we implemented.

In [6]:
result = newtons_method(grad_rosenbrock, hessian_rosenbrock, x0)
print("Optimal solution:", result)

Optimal solution: [1. 1.]


c:\Users\tkaria\OneDrive - Delft University of Technology\Teaching\CH3133-CP\computational_practicum_lecture_coding\.venv\Lib\site-packages\numpy\_core\numeric.py:457: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
